<a href="https://colab.research.google.com/github/biggity2bit/colab/blob/master/politifact_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [0]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NaiveBayes").getOrCreate()

In [15]:
# Upload my files
from google.colab import files
training_data = files.upload()

Saving politifact_titles.csv to politifact_titles.csv
Saving Total_clean.csv to Total_clean (1).csv


In [17]:
from pyspark import SparkFiles
new_train_df = spark.read.csv("Total_clean.csv", sep=",", header=True)

# Show dataframe
new_train_df.show()

+--------------------+-----+------+
|               title|label|status|
+--------------------+-----+------+
|As U.S. budget fi...|    0|  TRUE|
|U.S. military to ...|    0|  TRUE|
|Senior U.S. Repub...|    0|  TRUE|
|FBI Russia probe ...|    0|  TRUE|
|Trump wants Posta...|    0|  TRUE|
|White House, Cong...|    0|  TRUE|
|Trump says Russia...|    0|  TRUE|
|Factbox: Trump on...|    0|  TRUE|
|Trump on Twitter ...|    0|  TRUE|
|Alabama official ...|    0|  TRUE|
|Jones certified U...|    0|  TRUE|
|New York governor...|    0|  TRUE|
|Factbox: Trump on...|    0|  TRUE|
|Trump on Twitter ...|    0|  TRUE|
|Man says he deliv...|    0|  TRUE|
|Virginia official...|    0|  TRUE|
|U.S. lawmakers qu...|    0|  TRUE|
|Trump on Twitter ...|    0|  TRUE|
|U.S. appeals cour...|    0|  TRUE|
|Treasury Secretar...|    0|  TRUE|
+--------------------+-----+------+
only showing top 20 rows



In [18]:
# Create the Politifact testing df
new_test_df = spark.read.csv("politifact_titles.csv", sep=",", header=True)

# Show dataframe
new_test_df.show()

+--------------------+--------------------+--------------------+
|        articletitle|               label|              status|
+--------------------+--------------------+--------------------+
|"Says gloves shou...|                   0|                TRUE|
|New York is one o...|                   0|                TRUE|
|Says it's illegal...|                   0|                TRUE|
|"""Small trials""...|                   0|                TRUE|
|"Says Texas ""eit...|                   0|                TRUE|
|"North Carolina i...|                   0|                TRUE|
|"Virginia has ""l...|                   0|                TRUE|
|  """Until this week| they [OSHA] were...| OSHA is not spec...|
|     """The poor are|          ironically| the most likely ...|
|"""Go look at oth...|   started to reopen| and then they sa...|
|"""Africans livin...|                   0|                TRUE|
|"Republicans ""ha...| even in the midd...|                   0|
|"Before COVID-19,...| an

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/dataviz-curriculum/day_2/yelp_reviews.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("yelp_reviews.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+--------+--------------------+
|   class|                text|
+--------+--------------------+
|positive|Wow... Loved this...|
|negative|  Crust is not good.|
|negative|Not tasty and the...|
|positive|Stopped by during...|
|positive|The selection on ...|
|negative|Now I am getting ...|
|negative|Honeslty it didn'...|
|negative|The potatoes were...|
|positive|The fries were gr...|
|positive|      A great touch.|
|positive|Service was very ...|
|negative|  Would not go back.|
|negative|The cashier had n...|
|positive|I tried the Cape ...|
|negative|I was disgusted b...|
|negative|I was shocked bec...|
|positive| Highly recommended.|
|negative|Waitress was a li...|
|negative|This place is not...|
|negative|did not like at all.|
+--------+--------------------+
only showing top 20 rows



In [4]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['text']))
data_df.show()

+--------+--------------------+------+
|   class|                text|length|
+--------+--------------------+------+
|positive|Wow... Loved this...|    24|
|negative|  Crust is not good.|    18|
|negative|Not tasty and the...|    41|
|positive|Stopped by during...|    87|
|positive|The selection on ...|    59|
|negative|Now I am getting ...|    46|
|negative|Honeslty it didn'...|    37|
|negative|The potatoes were...|   111|
|positive|The fries were gr...|    25|
|positive|      A great touch.|    14|
|positive|Service was very ...|    24|
|negative|  Would not go back.|    18|
|negative|The cashier had n...|    99|
|positive|I tried the Cape ...|    59|
|negative|I was disgusted b...|    62|
|negative|I was shocked bec...|    50|
|positive| Highly recommended.|    19|
|negative|Waitress was a li...|    38|
|negative|This place is not...|    51|
|negative|did not like at all.|    20|
+--------+--------------------+------+
only showing top 20 rows



In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='class',outputCol='label')
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')


In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [0]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [0]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data_df)
cleaned = cleaner.transform(data_df)

In [10]:
# Show label and resulting features
cleaned.select(['label', 'features']).show(truncate=False)

+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                                                                                                |
+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262145,[33933,69654,123604,262144],[4.51085950651685,6.215607598755275,3.8642323415917974,24.0])                                                                             

In [0]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [12]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(5)

+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|   class|                text|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|negative|"I don't know wha...|    85|  1.0|["i, don't, know,...|["i, know, big, d...|(262144,[8478,300...|(262144,[8478,300...|(262145,[8478,300...|[-633.68462561935...|[5.70171928547901...|       1.0|
|negative|"It was extremely...|    51|  1.0|["it, was, extrem...|["it, extremely, ...|(262144,[7388,163...|(262144,[7388,163...|(262145,[7388,163...|[-463.68982706177...|[4.37393635174908.

In [13]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.674060
